In [ ]:
import os, shutil            #Operating System
#from pyswmm import Simulation, raingages, Nodes, SystemStats, Links
import pandas as pd
from swmmio.utils.modify_model import replace_inp_section
from swmmio import create_dataframeINP
import swmmio
from swmmio.graphics import swmm_graphics as sg
import math
import numpy as np
import datetime as DT
from functions import *

In [ ]:
InpFile = ! zenity --file-selection  --separator=, --file-filter=*.inp


In [ ]:
baseline = swmmio.Model(InpFile[0])
baseline.inp.path

# Creo nuova cartella di lavoro
workdir = baseline.inp.dir + '/' + baseline.inp.name
try: 
    os.mkdir(workdir)
except FileExistsError:
    pass
os.chdir(workdir)

In [ ]:
os.getcwd()

In [ ]:
# Richiama il solutore SWMM5
def swmm5solver (inpfile):
 #   if os.path.isfile(inpfile[:-4] + '.rpt'):
 #       print('Analisi non eseguita: ' + inpfile[:-4] + '.rpt' + ' esiste già')
 #       return
    print('Eeguo l\'analisi di ' + inpfile)
    inpfile = inpfile[:-4] # Nome file senza estensione
    
    if os.sys.platform == 'linux':
        swmmexepath = 'swmm51' ## Per windows bisogna inserire il percorso di swmm5.exe
        ! {swmmexepath} "{inpfile}.inp" "{inpfile}.rpt" "{inpfile}.out" > /dev/null 
#        ! {swmmexepath} "{inpfile}.inp" "{inpfile}.rpt"  > /dev/null 2> /dev/null #
        ! rm "{inpfile}.out" # i file out occupano molto  sppazio
        print('Creato file di report ' + inpfile + '.rpt')
    elif os.sys.platform == 'windows':
        #  bisogna aggiungere il percorso di swmm5.exe alla PATH di windows:
        #  https://www.wikihow.com/Run-a-Program-on-Command-Prompt [Metodo2]
        swmmexepath = 'C:\Program Files (x86)\EPA SWMM 5.1.013\swmm5.exe'
        ! {swmmexepath} "{inpfile}.inp" "{inpfile}.rpt" "{inpfile}.out"
        ! del "{inpfile}.out" # i file out occupano molto  sppazio
        print('Creato file di report ' + inpfile + '.rpt')
    else: 
        print('swmm5solver: Sistema operativo non supportato')


In [ ]:
inp2read = baseline.inp.path

# Definizione dei diametri commericiali
# https://www.oppo.it/tabelle/diametri_interni_tubi.htm#fibrocemento
DiametriCommericali = [.060,.080,.100,.125,
                       .150,.170,.200,.225,
                       .250,.300,.350,.400,
                       .450,.500,.600,.700,
                       .800,.900,1.00,1.10,1.20]

try: 
    os.mkdir('swmm' )
except FileExistsError:
    pass

for iteration in range(1,10): # 6 iterazioni
    
    print('\nIterazione N: ' + str(iteration) + '\n')
    
    try: 
        os.mkdir('./swmm/iter' + str(iteration) )
    except FileExistsError:
        pass
    
    try: 
        os.mkdir('./swmm/iter' + str(iteration + 1) )
    except FileExistsError:
        pass

    inp2verify = os.path.join('./swmm/iter' + str(iteration) + '/'+ \
                                   baseline.inp.name + '.inp')
    
    newfilebase = inp2verify[:-4] # Nome file senza estensione
    print(newfilebase)
    # Creo nuovo inp 
    try:
        shutil.copyfile(inp2read, inp2verify)
    except:
        pass
    
    # Plot dei diametri delle condotte
    plotGeomNetwork(inp2verify)

    # ESEGUO L'ANALISI Con SWMM5
    swmm5solver(inp2verify)
        
    mymodel = swmmio.Model(inp2verify)
    #print(mymodel.rpt_is_valid())
    nodes = mymodel.nodes()
    conds = mymodel.conduits()
    results = swmmio.create_dataframeRPT(inp2verify[:-4] + '.rpt')
    
  #  print(mymodel.conduits())

    # Plotto il riempimento delle Condotte e dei Nodi
    print('plot del grafico ', newfilebase + '_riempimento.png')
 #   plotModel(inpfilepath = inp2verify, 
 #             title = str(iteration), 
 #             annotation = 'Percentuale di riempimento\nnei nodi e nelle condotte',
 #             outfile = newfilebase + '_riempimento.png',
 #             Property_nodes = '',
 #             Property_conds = 'MaxDPerc',
 #             legend = False,
 #            )
        
    # inserisco i risultati delle analisi nei dataframe risultati
    
    df = pd.DataFrame()

    df['InElevation'] = list(nodes.InvertElev[conds.InletNode] )
    for link in df.index[df.InElevation.isnull()]: # fix Problema con i Dividers
        InNode = conds.InletNode[link]
        InElev = float(dividers.Elevation[InNode])
        # print(link, InNode, InElev)
        df.loc[link,'InElevation'] = InElev

    df['OutElevation'] = list(nodes.InvertElev[conds.OutletNode])
    for link in df.index[df.OutElevation.isnull()]: # fix Problema con i Dividers
        OutNode = conds.OutletNode[link]
        OutElev = float(dividers.Elevation[OutNode])
        # print(link, InNode, InElev)
        df.loc[link,'OutElevation'] = OutElev
    df.index = conds.index

    df['Length'] = conds.Length
    df['Pendenza'] = ( df.InElevation - df.OutElevation ) / df.Length

    b = 10 ** (-9/8) 

    df['MaxQ']= results.MaxQ.tolist()
    #df['idxmaxQ'] = df_MaxQ.T.idxmax()

    Qmax = df['MaxQ']
    Ks   = conds.ManningN ** -1
    i    = df.Pendenza

    TetaOpt = theta(.8)

    df['Dopt'] =  b * ( 2 ** (13/3) * Qmax / Ks / (i ** .5) *   \
                  (1 - math.sin(TetaOpt) / TetaOpt) ** (-2/3) * \
                  (TetaOpt - math.sin(TetaOpt)) ** -1 ) ** (3/8)

    df['Diametro'] = df.Dopt.apply(ceil2list, lst = DiametriCommericali)
    
    df.to_csv('./swmm/iter' + str(iteration) + '/'+ baseline.inp.name + '_Condotte.csv')
    
    dfConds = pd.DataFrame()
    dfConds['Shape'] = conds.Shape
    dfConds['Geom1'] = df.Diametro
    dfConds['Geom2'] = conds.Geom2
    dfConds['Geom3'] = conds.Geom3
    dfConds['Geom4'] = conds.Geom4
    dfConds['Barrels'] = conds.Barrels

    # Creo nuovo file di progetto con le nuove dimensioni per l'iterazione successiva
    
    inp2write = os.path.join('./swmm/iter' + str(iteration + 1) + '/'+ \
                                   baseline.inp.name + '.inp')
    
    shutil.copyfile(inp2verify, inp2write)
    replace_inp_section(inp2write, '[XSECTIONS]', dfConds)
    
    df.to_csv('./swmm/iter' + str(iteration) + '/'+ baseline.inp.name + '_CondotteNuove.csv')
    
    # Quante linee sono cambiate nel file inp?
    diff = ! diff {inp2verify} {inp2write} | wc -l
    
    diff = int(diff[0]) / 2
    
    print('Numero approssimativo condotte cambiate ~' + str(diff))
    
    if diff == 0:
        print('i file ' + \
             str(inp2verify) + \
             ' e ' + \
             str(inp2write) + \
             ' sono uguali!')
        break
    
    inp2read = inp2write

In [ ]:
Model = swmmio.Model('./swmm/iter3/MaysWenzel.inp')
CONDS = Model.conduits()
NODES = Model.nodes()

In [ ]:
Cost = (CONDS.Geom1.apply(Cp, H = 2.43) * CONDS.Length).sum() + NODES.MaxDepth.apply(Cm).sum()
print(Cost)

In [ ]:
CONDS.T

In [ ]:
results = swmmio.create_dataframeRPT(inp2verify[:-4] + '.rpt')

In [ ]:
results

In [ ]:
AAA.rpt


In [ ]:
os.chdir('swmm/iter2/')